In [2]:
import tensorflow as tf
from keras import __version__
tf.keras.__version__ = __version__
from imp import reload

import time
import random
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import clear_output

from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten, Embedding, Reshape

from rl.agents.dqn import DQNAgent
from rl.policy import EpsGreedyQPolicy
from rl.memory import SequentialMemory

from env_class import BatteryManagementEnv

plt.style.use("ggplot")

In [3]:
env = BatteryManagementEnv()
env.reset()

array([50.], dtype=float32)

In [4]:
print("Action Space {}".format(env.action_space))
print("State Space {}".format(env.observation_space))

Action Space 1
State Space 1


In [5]:
print(env.nA)

4


In [6]:
from keras.layers import Dense, Input
from keras.models import Model

#build neural network for DQN
def build_model(states, actions):
    input = Input(shape=(1,states))
    x = Flatten()(input)
    x = Dense(16, activation='relu')(x)
    #output layer
    output = Dense(actions, activation='linear')(x)

    model = Model(inputs=input, outputs=output)

    return model

model = build_model(env.observation_space, env.nA)

In [7]:
from rl.agents.dqn import DQNAgent
from rl.policy import BoltzmannQPolicy, EpsGreedyQPolicy, LinearAnnealedPolicy
from rl.memory import SequentialMemory
from keras.src.saving import serialization_lib
serialization_lib.enable_unsafe_deserialization()
from tensorflow.keras.optimizers.legacy import Adam

# Then, define DQN agent in Keras-RL
memory = SequentialMemory(limit=20000, window_length=1)
policy = LinearAnnealedPolicy(EpsGreedyQPolicy(), 
                              attr='eps', value_max=1., value_min=.1, value_test=.05, nb_steps=10000)
dqn = DQNAgent(model=model, nb_actions=env.nA, memory=memory, policy=policy,
                nb_steps_warmup=500, target_model_update=1e-2, enable_double_dqn=True, enable_dueling_network=True)
dqn.compile(optimizer=Adam(learning_rate=1e-3), metrics=['mae'])

In [13]:
dqn.fit(env, nb_steps=5000, visualize=False, verbose=1, nb_max_episode_steps=100, log_interval=1000)

Training for 5000 steps ...
Interval 1 (0 steps performed)
1000/1000 [==============================] - 5s 5ms/step - reward: 637.8000
1000 episodes - episode_reward: 637.800 [0.000, 900.000] - loss: 0.001 - mae: 452.583 - mean_q: 900.000 - mean_eps: 0.932

Interval 2 (1000 steps performed)
1000/1000 [==============================] - 10s 10ms/step - reward: 668.7000
1000 episodes - episode_reward: 668.700 [0.000, 900.000] - loss: 0.001 - mae: 453.994 - mean_q: 900.000 - mean_eps: 0.865

Interval 3 (2000 steps performed)
1000/1000 [==============================] - 11s 11ms/step - reward: 688.4000
1000 episodes - episode_reward: 688.400 [0.000, 900.000] - loss: 0.001 - mae: 454.532 - mean_q: 900.000 - mean_eps: 0.775

Interval 4 (3000 steps performed)
1000/1000 [==============================] - 11s 11ms/step - reward: 705.2000
1000 episodes - episode_reward: 705.200 [0.000, 900.000] - loss: 0.002 - mae: 453.258 - mean_q: 900.000 - mean_eps: 0.685

Interval 5 (4000 steps performed)
100